In [1]:
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from matplotlib import pyplot as plt


# Census API Key
c = Census(census_key, year=2013)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E", "B19301_001E","B17001_003E", "B17001_017E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B19301_001E": "Income Per Capita",
                                      "B17001_003E": "Men Living in Poverty",
                                      "B17001_017E": "Females Living in Poverty",
                                      "zip code tabulation area": "Zipcode"})

# Add in Employment Rate (Employment Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Income Per Capita","Poverty Rate", 
                       "Poverty Count", "Men Living in Poverty", "Females Living in Poverty"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Income Per Capita,Poverty Rate,Poverty Count,Men Living in Poverty,Females Living in Poverty
0,01832,22121.0,29014.0,10.903666,2412.0,1156.0,1256.0
1,01833,8295.0,43297.0,2.302592,191.0,88.0,103.0
2,01834,6675.0,40222.0,2.187266,146.0,53.0,93.0
3,01835,13527.0,31429.0,8.523693,1153.0,560.0,593.0
4,01840,4547.0,14758.0,39.542556,1798.0,696.0,1102.0


In [3]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
census_data_orig = pd.read_csv("Resources/zip_bank_data.csv", dtype="object", encoding="utf-8")

# Visualize
census_data_orig.head()

,Zipcode,Address,Median Age,Household Income,Per Capita Income,Lat,Lng,Bank Count
0,624,"Hastings, MI 49058, USA",40.8,46777,22137,42.6306916,-85.2929384,9
1,692,"Ball, LA 71405, USA",35.8,55242,23941,31.4061799,-92.396174,12
2,730,"Great Mills, MD 20634, USA",31.9,79944,35961,38.2201614,-76.4967919,9
3,757,"Williamsport, TN 38487, USA",41.6,38125,18884,35.7310368,-87.2419299,0
4,957,"Marion, ND 58466, USA",44.5,69844,36981,46.5594224,-98.3481542,1


In [4]:
# Merge the two data sets along zip code
census_data_complete = pd.merge(
    census_data_orig, census_pd, how="left", on=["Zipcode", "Zipcode"])

# Remove rows missing data
census_data_complete = census_data_complete.dropna()

# Visualize
census_data_complete.head()

,Zipcode,Address,Median Age,Household Income,Per Capita Income,Lat,Lng,Bank Count,Population,Income Per Capita,Poverty Rate,Poverty Count,Men Living in Poverty,Females Living in Poverty
63,10007,"Flatonia, TX 78941, USA",45.7,40304,23115,29.7574106,-97.1574784,0,6525.0,157713.0,2.206897,144.0,61.0,83.0
64,10303,"Nevada, IA 50201, USA",40.4,56619,28908,42.065743,-93.4599326,7,24537.0,22077.0,21.828259,5356.0,2334.0,3022.0
65,10309,"Lukachukai, AZ 86507, USA",24.1,22009,8346,36.4106866,-109.2593642,0,32646.0,35719.0,6.766526,2209.0,908.0,1301.0
66,10553,"Lone Pine, CA 93545, USA",40.6,32473,18444,36.5131184,-118.0888578,0,9895.0,26137.0,15.260232,1510.0,620.0,890.0
67,10803,"Niagara, WI 54151, USA",45.7,45813,23500,45.715354,-87.9804239,2,12439.0,73808.0,3.006673,374.0,122.0,252.0


In [5]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True) 
zipcode = search.by_zipcode("92688")
zipcode

SimpleZipcode(zipcode='92688', zipcode_type='Standard', major_city='Rancho Santa Margarita', post_office_city='Rancho Santa Margarita, CA', common_city_list=['Rancho Santa Margarita', 'Rancho Sta Marg', 'Rcho Sta Marg', 'Rsm'], county='Orange County', state='CA', lat=33.63, lng=-117.61, timezone='Pacific', radius_in_miles=4.0, area_code_list=['949'], population=43792, population_density=4018.0, land_area_in_sqmi=10.9, water_area_in_sqmi=0.04, housing_units=16120, occupied_housing_units=15530, median_home_value=474800, median_household_income=98821, bounds_west=-117.636191, bounds_east=-117.57645, bounds_north=33.672324, bounds_south=33.55156)